# Score Analysis 

I made a score analysis of the students' reading examination. The reading questions consist of inference, direct and indirect, main ideas, and vocabulary. 
To do the test, I used Google form. As for the result, I downloaded the file in .csv. Here is it looks like (The students' name are covered):
<img src = 'reading1.png' width = '1500' height= '500'>

1. I opened the .csv file
2. I dropped Timestamp and Score because I did not need the two columns. I need the question to be checked one by one.

In [ ]:
import pandas as pd
read = pd.read_csv('reading.csv')
read = read.drop(['Timestamp','Score'], axis = 1)


This is what the data looks like after the two columns dropped.
<img src = 'reading2.png' width = '1000' height= '1000'>

In [ ]:
#read.rename(columns={x:y for x,y in zip(read.columns,range(0,len(read.columns)))})
#change the whole columns name to numbers
read.columns = [str(i) for i in range(len(read.columns))]
read.columns

In [ ]:
import numpy as np

  
col_num = read.iloc[:,1:51]
read_1 = col_num.apply(lambda c: c.str.extract('\((.*?)\)', expand=False))
#read.col_num= read_1
read_clean = pd.concat([read['0'], read_1], axis = 1)


In [ ]:
#group the lesson details
infer = read_clean[['2', '4', '19', '25', '26', '33', '36', '44']]
infer_ans = ['D', 'A', 'D', 'B', 'D', 'C', 'D', 'A']

direct_qs = read_clean[['6', '8', '10', '15', '17', '21', '24', '34', '37', '39', '41', '42', '47']]
direct_ans = ['C', 'B', 'D', 'C', 'A', 'A', 'A', 'B', 'C', 'D', 'B', 'B', 'C']
main_ideas = read_clean[['1', '11', '13', '20', '30', '40', '50']]
main_ideas_ans = ['C','B', 'D', 'B', 'D', 'D', 'C']

vocab = read_clean[['5', '7', '9', '12', '14', '16', '18', '22', '23', '27', '28', '31', '32', '35','38', '43', '45','46', '48']]
vocab_ans = ['B', 'D', 'A', 'B', 'A', 'C', 'B', 'D', 'A', 'A', 'D', 'B', 'A', 'A', 'B', 'C', 'D', 'B', 'C']

''' check whether the answers are correct or not and sum the amount'''
infer_correct = infer == infer_ans
read_clean['infer_total'] = infer_correct.sum(axis = 1)

direct_qs_correct = direct_qs == direct_ans
read_clean['direct_qs'] = direct_qs_correct.sum(axis = 1)

main_ideas_correct = main_ideas == main_ideas_ans
read_clean['main_ideas'] = main_ideas_correct.sum(axis = 1)

vocab_correct = vocab == vocab_ans
read_clean['vocabs'] = vocab_correct.sum(axis = 1)


all_true_des = [' was excellent', ' was very good', 'scored well'] 
half_true = [' was decent', ' was good', ' was average', 'was so-so']
not_really = [' needed improvement', ' needed a lof of practice', ' needed practice more']



In [ ]:
import random

#infer_total: 6-8(all_true), 3-5 (half_true), 0-2(not_really)
def set_desc(row):
    if row["infer_total"] < 3:
        return random.choice(not_really)
    elif row["infer_total"] < 6:
        return random.choice(half_true)
    elif row["infer_total"] <= 8:
        return random.choice(all_true_des)

#direct: 13
def direct(row):
    if row['direct_qs'] <= 4:
        return random.choice(not_really)
    elif row['direct_qs'] >= 5 and row['direct_qs'] <= 9:
        return random.choice(half_true )
    elif row['direct_qs'] <= 13:
        return random.choice(all_true_des)

def main(row): #7 
    if row["main_ideas"] < 2:
        return random.choice(not_really)
    elif row["main_ideas"] <= 7:
        return random.choice(all_true_des)
    elif row["main_ideas"] < 5:
        return random.choice(half_true)

# vocab : 14-19(all_true), 7-13 (half_true), 0-6(not_really)

def vocab(row):
    if row['vocabs'] <= 6:
        return random.choice(not_really)
    elif row['vocabs'] >= 7 and row['vocabs'] <= 13:
        return random.choice(half_true )
    elif row['vocabs'] <= 19:
        return random.choice(all_true_des)

read_clean = read_clean.assign(infer_desc=read_clean.apply(set_desc, axis=1))
read_clean = read_clean.assign(direct_qs_desc=read_clean.apply(direct, axis=1))
read_clean = read_clean.assign(main_desc=read_clean.apply(main, axis=1))
#check lagi main idea samo vocab
read_clean = read_clean.assign(vocab_desc=read_clean.apply(vocab, axis=1))

In [ ]:

        
for (student, infer, direct, main, vocab) in zip(read_clean['0'], read_clean['infer_desc'], read_clean['direct_qs_desc'], read_clean['main_desc'],read_clean['vocab_desc'] ): 
    print(f'When it comes to reading, {student}, the abilities to answer inference questions {infer}, direct and indirect questions {direct}, main idea questions {main}, and vocabulary questions {vocab}')
    